In [1]:
# Import all necessary packages

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import math
from sklearn import preprocessing

In [2]:
# Set up GPU accelerated training

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('using GPU:', torch.cuda.get_device_name()) if torch.cuda.is_available() else 'using cpu'

'using cpu'

# Data generation

In [3]:
# specifying constant parameters

T_0 = 300
V = 1
k_0 = 8.46*(np.power(10,6))
C_p = 0.231
rho_L = 1000
Q_s = 0.0
T_s = 402
F = 5
E = 5*(np.power(10,4))
delta_H = -1.15*(np.power(10,4))
R = 8.314
C_A0s = 4
C_As = 1.95
t_final = 0.005
t_step = 1e-4
P = np.array([[1060, 22], [22, 0.52]])

In [10]:
def generate_new_1000(x):
    return x
    # return x + x * np.random.uniform(-10, 10)

def generate_new_100(x):
    return x
    # return x + x * np.random.uniform(-1, 1)

def generate_new_5(x):
    return x
    # return x + x * np.random.uniform(-0.05, 0.05)

T_0_new = generate_new_100(T_0)
V_new = generate_new_1000(V)
F_new = generate_new_1000(F)
C_A0s_new = generate_new_100(C_A0s)
Q_s_new = generate_new_1000(Q_s)
rho_L_new = generate_new_5(rho_L)
C_p_new = generate_new_5(C_p)
k_0_new = generate_new_5(k_0)
E_new = generate_new_5(E)
delta_H_new = generate_new_5(delta_H)

In [11]:
# generating inputs and initial states for CSTR, all expressed in deviation form

u1_list = np.linspace(-3.5, 3.5, 4, endpoint=True)
u2_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
T_initial = np.linspace(300, 600, 50, endpoint=True) - T_s
CA_initial = np.linspace(0, 6, 50, endpoint=True) - C_As

# u1_list = np.linspace(-0.5, 0.5, 2, endpoint=True)
# u2_list = np.linspace(-5e1, 5e1, 2, endpoint=True)
# T_initial = np.linspace(380, 420, 5, endpoint=True) - T_s
# CA_initial = np.linspace(0, 2, 5, endpoint=True) - C_As

In [12]:
# sieve out initial states that lie outside of stability region

T_start = list()
CA_start = list()

for T in T_initial:
    for CA in CA_initial:
        x = np.array([CA, T])
        if x @ P @ x < 372:
          CA_start.append(CA)
          T_start.append(T)
print("number of initial conditions: {}".format(len(CA_start)))

# convert to np.arrays
CA_start = np.array([CA_start])
T_start = np.array([T_start])
x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)  # every row is a pair of initial states within stability region
print("shape of x_deviation is {}".format(x_deviation.shape))

number of initial conditions: 190
shape of x_deviation is (190, 2)


In [13]:
# Open-loop simulations of the first-principles model of CSTR

def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial + C_As
    T = T_initial + T_s

    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)

        C_A += dCAdt * t_step
        T += dTdt * t_step

        if (i+1)% 5 == 0:
            C_A_list.append(C_A - C_As)  # in deviation form
            T_list.append(T - T_s)  # in deviation form


    return C_A_list, T_list

In [14]:
# get X and y data for training and testing

CA_output = list()
T_output = list()
CA_input = list()
T_input = list()
CA0_input = list()
Q_input = list()

for u1 in u1_list:
    C_A0 = u1 + C_A0s_new

    for u2 in u2_list:
        Q = u2 + Q_s_new

        for C_A_initial, T_initial in x_deviation:
            C_A_list, T_list = \
                CSTR_simulation(F_new, V_new, C_A0, k_0_new, E_new, R, T_0_new, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)

            if np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                CA0_input.append(u1)
                Q_input.append(u2)
                CA_input.append(C_A_initial)
                T_input.append(T_initial)

                CA_output.append(C_A_list)
                T_output.append(T_list)

In [15]:
# collate input for RNN

CA0_input = np.array(CA0_input)
CA0_input = CA0_input.reshape(-1,1,1)

Q_input = np.array(Q_input)
Q_input = Q_input.reshape(-1,1,1)

CA_input = np.array(CA_input)
CA_input = CA_input.reshape(-1,1,1)

T_input = np.array(T_input)
T_input = T_input.reshape(-1,1,1)

RNN_input = np.concatenate((CA_input, T_input, CA0_input, Q_input), axis=2)

"""
    the input to RNN is in the shape [number of samples x timestep x variables], and the input variables are same for every
    time step, not sure if my treatment here is correct
"""
RNN_input = RNN_input.repeat(10, axis=1) # 10 time steps in this example
print("RNN_input shape is {}".format(RNN_input.shape))

# checking the input is duplicated 10 times for each time step
print(RNN_input[0, 0])
print(RNN_input[0, 1])

RNN_input shape is (3040, 10, 4)
[ 1.35612245e+00 -7.13877551e+01 -3.50000000e+00 -5.00000000e+05]
[ 1.35612245e+00 -7.13877551e+01 -3.50000000e+00 -5.00000000e+05]


In [16]:
# collate output for RNN

CA_output = np.array(CA_output)
CA_output = CA_output.reshape(-1, 10, 1)

T_output = np.array(T_output)
T_output = T_output.reshape(-1, 10, 1)

RNN_output = np.concatenate((CA_output, T_output), axis=2)
print("RNN_output shape is {}".format(RNN_output.shape))  # output shape: number of samples x timestep x variables

# checking output
print('RNN_output shape is',RNN_output.shape)

RNN_output shape is (3040, 10, 2)
RNN_output shape is (3040, 10, 2)


In [17]:
# Normalization

scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, 4))
scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

print(scaler_X.mean_)
print(scaler_X.var_)
print(scaler_y.mean_)
print(scaler_y.var_)

RNN_input = scaler_X.transform(RNN_input.reshape(-1, 4)).reshape(-1,10,4)
RNN_output = scaler_y.transform(RNN_output.reshape(-1, 2)).reshape(-1,10,2)

# split into train and test sets

# X_train, X_test, y_train, y_test = train_test_split(RNN_input, RNN_output, test_size=0.3, random_state=123)
# # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=123)
# X_collocation_train, X_DataDriven_train, y_collocation_train, y_DataDriven_test = train_test_split(X_train, y_train, test_size=0.01, random_state=123)
# X_collocation_train, X_collocation_val, y_collocation_train, y_collocation_val = train_test_split(X_collocation_train, y_collocation_train, test_size=0.3, random_state=123)
# X_DataDriven_train, X_DataDriven_val, y_DataDriven_train, y_DataDriven_val = train_test_split(X_DataDriven_train, y_DataDriven_test, test_size=0.3, random_state=123)

# checking X_train
# print(X_train[0, 0])
# print(X_train[0, 1])

mean_y = torch.from_numpy(scaler_y.mean_).float()
std_y = torch.from_numpy(np.sqrt(scaler_y.var_)).float()

isCorrect = False
if np.isnan(RNN_input).any() == False and np.isnan(RNN_output).any() == False and np.isinf(RNN_input).any() == False and np.isinf(RNN_output).any() == False and any(abs(i) > 10 for i in RNN_output.reshape(-1)) == False:
  isCorrect = True

print(isCorrect)

[ 7.25026853e-03 -3.35123523e-01  1.19670355e-16 -3.92135821e-12]
[7.01467140e-01 1.41521192e+03 6.80555556e+00 1.38888889e+11]
[ 0.01052708 -0.49783967]
[6.93259332e-01 1.43858455e+03]
True


In [18]:
X_DataDriven_train = torch.from_numpy(RNN_input[:10]).float()
y_DataDriven_train = torch.from_numpy(RNN_output[:10]).float()

dataset_DataDriven_train = TensorDataset(X_DataDriven_train,y_DataDriven_train)
dataloader_DataDriven_train = DataLoader(dataset_DataDriven_train, batch_size=256, shuffle=True)

torch.save(dataloader_DataDriven_train,'dataloader_DataDriven_train.pt')

In [19]:
X_total_test = torch.from_numpy(RNN_input).float()
y_total_test = torch.from_numpy(RNN_output).float()

dataset_total_test = TensorDataset(X_total_test,y_total_test)
dataloader_total_test = DataLoader(dataset_total_test, batch_size=6400, shuffle=False)

torch.save(dataloader_total_test,'dataloader_total_test.pt')

In [20]:
# generate collocation points
seed = 0
rng = np.random.RandomState(seed)
sample_idx = rng.choice(len(RNN_input), size=500)
x_collocation = RNN_input[sample_idx]

X_collocation_train = torch.from_numpy(x_collocation).float()
torch.save(mean_y,'mean_y.pt')
torch.save(std_y,'std_y.pt')
np.save('mean_X.npy',scaler_X.mean_)
np.save('std_X.npy',np.sqrt(scaler_X.var_))

In [43]:
'''
Convert data to torch format
'''
# X_collocation_train = torch.from_numpy(X_collocation_train).float()
# X_collocation_val = torch.from_numpy(X_collocation_val).float()
# X_DataDriven_train = torch.from_numpy(X_DataDriven_train).float()
# X_DataDriven_val = torch.from_numpy(X_DataDriven_val).float()

# y_collocation_train = torch.from_numpy(y_collocation_train).float()
# y_collocation_val = torch.from_numpy(y_collocation_val).float()
# y_DataDriven_train = torch.from_numpy(y_DataDriven_train).float()
# y_DataDriven_val = torch.from_numpy(y_DataDriven_val).float()


# X_total_test = torch.from_numpy(X_test).float()
# y_total_test = torch.from_numpy(y_test).float()


print(f'X_collocation_train shape is: {X_collocation_train.shape}')
# print(f'y_collocation_train shape is: {y_collocation_train.shape}')
# print(f'X_collocation_val shape is: {X_collocation_val.shape}')
# print(f'y_collocation_val shape is: {y_collocation_val.shape}')

print(f'X_DataDriven_train shape is: {X_DataDriven_train.shape}')
print(f'y_DataDriven_train shape is: {y_DataDriven_train.shape}')
# print(f'X_DataDriven_val shape is: {X_DataDriven_val.shape}')
# print(f'y_DataDriven_val shape is: {y_DataDriven_val.shape}')

print(f'X_total_test shape is: {X_total_test.shape}')
print(f'y_toal_test shape is: {y_total_test.shape}')


X_collocation_train shape is: torch.Size([500, 10, 4])
X_DataDriven_train shape is: torch.Size([1, 10, 4])
y_DataDriven_train shape is: torch.Size([1, 10, 2])
X_total_test shape is: torch.Size([3040, 10, 4])
y_toal_test shape is: torch.Size([3040, 10, 2])


In [ ]:
'''
Creating data loaders for training, validation, and testing datasets
'''
# dataset_DataDriven_train = TensorDataset(X_DataDriven_train,y_DataDriven_train)
# dataloader_DataDriven_train = DataLoader(dataset_DataDriven_train, batch_size=256, shuffle=True)

# dataset_DataDriven_val = TensorDataset(X_DataDriven_val,y_DataDriven_val)
# dataloader_DataDriven_val = DataLoader(dataset_DataDriven_val, batch_size=256, shuffle=True)

# dataset_total_test = TensorDataset(X_total_test,y_total_test)
# dataloader_total_test = DataLoader(dataset_total_test, batch_size=256, shuffle=False)

'''
Save data loaders of training, validation, and testing datasets
'''
# torch.save(dataloader_DataDriven_train,'dataloader_DataDriven_train.pt')
# # torch.save(dataloader_DataDriven_val,'dataloader_DataDriven_val.pt')
# torch.save(dataloader_total_test,'dataloader_total_test.pt')

# torch.save(mean_y,'mean_y.pt')
# torch.save(std_y,'std_y.pt')
# np.save('mean_X.npy',scaler_X.mean_)
# np.save('std_X.npy',np.sqrt(scaler_X.var_))

# Training process

In [25]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [21]:
class RNN(nn.Module):
    "Defines a RNN network"

    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super(RNN, self).__init__()
        self.layers = N_LAYERS

        if isinstance(N_HIDDEN, list):
            self.rnn = nn.RNN(N_INPUT,
                              N_HIDDEN[0],
                              batch_first=True,
                              nonlinearity='relu')

            self.rnn1 = nn.ModuleList(
                [nn.RNN(N_HIDDEN[i],
                        N_HIDDEN[i+1],
                        batch_first=True,
                        nonlinearity='relu') for i in range(N_LAYERS - 1)]
            )

            self.output_layer = nn.Linear(N_HIDDEN[-1], N_OUTPUT)

            self.list_flag = True

        else:
            self.rnn = nn.RNN(N_INPUT,
                              N_HIDDEN,
                              N_LAYERS,
                              batch_first=True,
                              nonlinearity='relu')

            self.output_layer = nn.Linear(N_HIDDEN, N_OUTPUT)

            self.list_flag = False

    def forward(self, x):
        x, _ = self.rnn(x)

        if self.list_flag:
            for i in range(self.layers - 1):
                x, _ = self.rnn1[i](x)

        x = self.output_layer(x)
        return x

In [22]:
mean_X = np.load('mean_X.npy')
std_X = np.load('std_X.npy')
mean_y = torch.load('mean_y.pt')
std_y=torch.load('std_y.pt')

In [23]:
def train_model(model,n_epochs,X_collocation_train):

    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = []

    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3) # Adam optimizer

    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=50, verbose=True)

    for epoch in range(1,n_epochs+1):
        ###################
        # train the model #
        ###################
        model.train() # prep model for training

        for id_batch, (x_batch, y_batch) in enumerate(dataloader_DataDriven_train):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            NN_output = model(x_batch) # the predicted values x̃ are obtained by passing the input vector to the PIRNN model

            # Data driven loss term
            loss1 = torch.mean((NN_output[:, :, :] - y_batch[:, :, :])**2)  # use mean squared error

            X_collocation_train = X_collocation_train.to(device)  # 13100 sample
            NN_output = model(X_collocation_train)

            # Compute the physics-driven loss term
            CA_NN_input = X_collocation_train[:, :, 0] * std_X[0] + mean_X[0] + C_As
            T_NN_input = X_collocation_train[:, :, 1] * std_X[1] + mean_X[1] + T_s
            C_A0 = X_collocation_train[:, :, 2] * std_X[2] + mean_X[2] + C_A0s_new
            Q = X_collocation_train[:, :, 3] * std_X[3] + mean_X[3] + Q_s_new

            NN_output = NN_output * std_y.to(device) + mean_y.to(device) + torch.from_numpy(np.array([C_As, T_s])).float().to(device)

            dCA_first = (NN_output[:, 1:2, 0] - CA_NN_input[:, 0:1 ]) / (2*t_step*5)
            dT_first = (NN_output[:, 1:2, 1] - T_NN_input[:, 0:1]) / (2*t_step*5)

            dCA_center = (NN_output[:, 2:, 0] - NN_output[:, :-2, 0]) / (2*t_step*5)
            dT_center = (NN_output[:, 2:, 1] - NN_output[:, :-2, 1]) / (2*t_step*5)

            dCA_last = (NN_output[:, -1:, 0] - NN_output[:, -2:-1, 0]) / (t_step*5)
            dT_last = (NN_output[:, -1:, 1] - NN_output[:, -2:-1, 1]) / (t_step*5)


            dCA = torch.cat((dCA_first, dCA_center, dCA_last), 1)
            dT = torch.cat((dT_first, dT_center, dT_last), 1)

            lossCA = dCA - F_new / V_new * (C_A0 - NN_output[:, :, 0]) + k_0 * torch.exp(-E / (R * NN_output[:, :, 1])) * NN_output[:, :, 0]**2
            lossCA = torch.mean(lossCA**2)

            lossT = dT - F_new / V_new * (T_0_new - NN_output[:, :, 1]) + delta_H / (rho_L * C_p) * k_0 * torch.exp(-E / (R * NN_output[:, :, 1])) * NN_output[:, :, 0]**2 - Q / (rho_L * C_p * V_new)
            lossT = torch.mean(lossT**2)

            # print(1e2 * loss1, 1e-1 * lossCA, 1e-5 * lossT)
            # backpropagate joint loss
            loss = 1e2 * loss1 + 1e-1 * lossCA + 1e-5 * lossT # add all loss terms together

            loss.backward()
            optimizer.step()
            # record training loss
            train_losses.append(loss.item())

        # train_loss = np.average(train_losses)
        # epoch_len = len(str(n_epochs))
        # print_msg = (f'[{epoch}/{n_epochs}] ' +
        #               f'train_loss: {train_loss:.5f} ')
        # print(print_msg)

    #     ######################
    #     # validate the model #
    #     ######################
    #     model.eval() # prep model for evaluation
    #     for val_batch,(x_valbatch, y_valbatch) in enumerate(dataloader_DataDriven_val):
    #         x_valbatch, y_valbatch = x_valbatch.to(device), y_valbatch.to(device)  # use valiadation data
    #         NN_output = model(x_valbatch)
    #         # Data driven loss term

    #         loss1 = torch.mean((NN_output[:, :, :] - y_valbatch[:, :, :])**2)

    #         X_collocation_val = X_collocation_val.to(device)
    #         NN_output = model(X_collocation_val)

    #         # Compute the physics-driven loss term
    #         CA_NN_input = X_collocation_val[:, :, 0] * std_X[0] + mean_X[0] + C_As
    #         T_NN_input = X_collocation_val[:, :, 1] * std_X[1] + mean_X[1] + T_s
    #         C_A0 = X_collocation_val[:, :, 2] * std_X[2] + mean_X[2] + C_A0s
    #         Q = X_collocation_val[:, :, 3] * std_X[3] + mean_X[3] + Q_s

    #         NN_output = NN_output * std_y.to(device) + mean_y.to(device) + torch.from_numpy(np.array([C_As, T_s])).float().to(device)

    #         dCA_first = (NN_output[:, 1:2, 0] - CA_NN_input[:, 0:1]) / (2*t_step*5)
    #         dT_first = (NN_output[:, 1:2, 1] - T_NN_input[:, 0:1]) / (2*t_step*5)


    #         dCA_center = (NN_output[:, 2:, 0] - NN_output[:, :-2, 0]) / (2*t_step*5)
    #         dT_center = (NN_output[:, 2:, 1] - NN_output[:, :-2, 1]) / (2*t_step*5)

    #         dCA_last = (NN_output[:, -1:, 0] - NN_output[:, -2:-1, 0]) / (t_step*5)
    #         dT_last = (NN_output[:, -1:, 1] - NN_output[:, -2:-1, 1]) / (t_step*5)


    #         dCA = torch.cat((dCA_first, dCA_center, dCA_last), 1)
    #         dT = torch.cat((dT_first, dT_center, dT_last), 1)

    #         lossCA = dCA - F / V * (C_A0 - NN_output[:, :, 0]) + k_0 * torch.exp(-E / (R * NN_output[:, :, 1])) * NN_output[:, :, 0]**2
    #         lossCA = torch.mean(lossCA**2)

    #         lossT = dT - F / V * (T_0 - NN_output[:, :, 1]) + delta_H / (rho_L * C_p) * k_0 * torch.exp(-E / (R * NN_output[:, :, 1])) * NN_output[:, :, 0]**2 - Q / (rho_L * C_p * V)
    #         lossT = torch.mean(lossT**2)

    #         # backpropagate joint loss
    #         loss = 1e2 * loss1 + 1e-1 * lossCA + 1e-5 * lossT # add all loss terms together

    #         valid_losses.append(loss.item())

    #     # print training/validation statistics
    #     # calculate average loss over an epoch
    #     train_loss = np.average(train_losses)
    #     valid_loss = np.average(valid_losses)
    #     avg_train_losses.append(train_loss)
    #     avg_valid_losses.append(valid_loss)

    #     epoch_len = len(str(n_epochs))

    #     print_msg = (f'[{epoch}/{n_epochs}] ' +
    #                  f'train_loss: {train_loss:.5f} ' +
    #                  f'valid_loss: {valid_loss:.5f}')

    #     print(print_msg)
    #     # clear lists to track next epoch
    #     train_losses = []
    #     valid_losses = []

    #     # early_stopping needs the validation loss to check if it has decresed,
    #     # and if it has, it will make a checkpoint of the current model
    #     early_stopping(valid_loss, model)

    #     if early_stopping.early_stop:
    #         print("Early stopping")
    #         break

    # # load the last checkpoint with the best model
    # model.load_state_dict(torch.load('checkpoint.pt'))

    return  model, avg_train_losses, avg_valid_losses

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [620]:
from keras.layers import Dense, SimpleRNN, LSTM
import tensorflow as tf
from keras import Model, regularizers, activations
import pickle

class Model(tf.keras.layers.Layer):

    def __init__(self):
        super(Model, self).__init__()

        self.layer_1 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_2 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_3 = Dense(2, activation='linear')

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        x = self.layer_3(x)
        return x

model = Model()

# Load the Keras model from the pickle file
with open('/content/drive/MyDrive/Meta-Learning/CSTR+Batach+PFR/model_reptile_cstr_batch_pfr.sav', 'rb') as f:
    keras_model = pickle.load(f)

weights = keras_model.get_weights()
for item in weights:
  print(np.array(item).shape)

(4, 64)
(64, 64)
(64,)
(64, 64)
(64, 64)
(64,)
(64, 2)
(2,)


In [501]:
model_PINN = RNN(4, 2, [64, 64], 2)
for name, param in model_PINN.named_parameters():
  print(name)
  print(param.shape)


rnn.weight_ih_l0
torch.Size([64, 4])
rnn.weight_hh_l0
torch.Size([64, 64])
rnn.bias_ih_l0
torch.Size([64])
rnn.bias_hh_l0
torch.Size([64])
rnn1.0.weight_ih_l0
torch.Size([64, 64])
rnn1.0.weight_hh_l0
torch.Size([64, 64])
rnn1.0.bias_ih_l0
torch.Size([64])
rnn1.0.bias_hh_l0
torch.Size([64])
output_layer.weight
torch.Size([2, 64])
output_layer.bias
torch.Size([2])


In [389]:
# print(model_PINN.rnn.weight_ih_l0)
# model_PINN.rnn.weight_ih_l0 = torch.nn.Parameter(torch.tensor(weights[0].T))
# print(model_PINN.rnn.weight_ih_l0)

# Assign the weights to the PyTorch model
with torch.no_grad():
    # Assign kernel weights
    model_PINN.rnn.weight_ih_l0.data = torch.tensor(weights[0].T)
    # Assign recurrent weights
    model_PINN.rnn.weight_hh_l0.data = torch.tensor(weights[1].T)
    # Assign bias
    model_PINN.rnn.bias_hh_l0.data = torch.tensor(weights[2])
    # Assign bias
    model_PINN.rnn.bias_ih_l0.fill_(0)
    # Assign kernel weights
    model_PINN.rnn1[0].weight_ih_l0.data = torch.tensor(weights[3].T)
    # Assign recurrent weights
    model_PINN.rnn1[0].weight_hh_l0.data = torch.tensor(weights[4].T)
    # Assign bias
    model_PINN.rnn1[0].bias_hh_l0.data = torch.tensor(weights[5])
    # Assign bias
    model_PINN.rnn1[0].bias_ih_l0.fill_(0)
    # Assign output weight
    model_PINN.output_layer.weight.data = torch.tensor(weights[6].T)
    # Assign output bias
    model_PINN.output_layer.bias.data = torch.tensor(weights[7])

# print(model_PINN.rnn.bias_ih_l0)
# print(model_PINN.rnn.bias_hh_l0)

In [40]:
for i in range(1):

    seed = 0
    rng = np.random.RandomState(seed)
    sample_idx = rng.choice(len(RNN_input), size=1)
    x_train = RNN_input[sample_idx]
    y_train = RNN_output[sample_idx]

    X_DataDriven_train = torch.from_numpy(x_train).float()
    y_DataDriven_train = torch.from_numpy(y_train).float()

    dataset_DataDriven_train = TensorDataset(X_DataDriven_train,y_DataDriven_train)
    dataloader_DataDriven_train = DataLoader(dataset_DataDriven_train, batch_size=256, shuffle=True)

    model_PINN = RNN(4, 2, [64, 64], 2)

    # # Assign the weights to the PyTorch model
    # with torch.no_grad():
    #     # Assign kernel weights
    #     model_PINN.rnn.weight_ih_l0.data = torch.tensor(weights[0].T)
    #     # Assign recurrent weights
    #     model_PINN.rnn.weight_hh_l0.data = torch.tensor(weights[1].T)
    #     # Assign bias
    #     model_PINN.rnn.bias_hh_l0.data = torch.tensor(weights[2])
    #     # Assign bias
    #     model_PINN.rnn.bias_ih_l0.fill_(0)
    #     # Assign kernel weights
    #     model_PINN.rnn1[0].weight_ih_l0.data = torch.tensor(weights[3].T)
    #     # Assign recurrent weights
    #     model_PINN.rnn1[0].weight_hh_l0.data = torch.tensor(weights[4].T)
    #     # Assign bias
    #     model_PINN.rnn1[0].bias_hh_l0.data = torch.tensor(weights[5])
    #     # Assign bias
    #     model_PINN.rnn1[0].bias_ih_l0.fill_(0)
    #     # Assign output weight
    #     model_PINN.output_layer.weight.data = torch.tensor(weights[6].T)
    #     # Assign output bias
    #     model_PINN.output_layer.bias.data = torch.tensor(weights[7])

    # Train the PIRNN model
    model_PINN.to(device)
    print(model_PINN)

    n_epochs = 500
    model_PINN, train_loss, valid_loss = train_model(model_PINN, n_epochs, X_collocation_train)

    y_test_error = list()
    model_PINN.eval()
    for id_batch, (x_batch, y_batch) in enumerate(dataloader_total_test):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        NN_output = model_PINN(x_batch)
        MSE_loss= torch.mean((NN_output - y_batch)**2)
        y_test_error.append(MSE_loss.item())

    print(f"mean error is {np.mean(y_test_error)}, std is {np.std(y_test_error)}")

RNN(
  (rnn): RNN(4, 64, batch_first=True)
  (rnn1): ModuleList(
    (0): RNN(64, 64, batch_first=True)
  )
  (output_layer): Linear(in_features=64, out_features=2, bias=True)
)
mean error is 0.0007220251718536019, std is 0.0


In [41]:
torch.save(model_PINN,'PINN.pkl')

# Test PIRNN by MSE loss

In [42]:
"""
Test PIRNN model
"""
model_PINN = torch.load('PINN.pkl')
model_PINN.to(device)
print(model_PINN)

y_test_error = list()
model_PINN.eval()
for id_batch, (x_batch, y_batch) in enumerate(dataloader_total_test):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    NN_output = model_PINN(x_batch)
    MSE_loss= torch.mean((NN_output - y_batch)**2)
    y_test_error.append(MSE_loss.item())

print(f"mean error is {np.mean(y_test_error)}, std is {np.std(y_test_error)}")

RNN(
  (rnn): RNN(4, 64, batch_first=True)
  (rnn1): ModuleList(
    (0): RNN(64, 64, batch_first=True)
  )
  (output_layer): Linear(in_features=64, out_features=2, bias=True)
)
mean error is 0.0007220251718536019, std is 0.0


# Data-driven

In [29]:

def train_DataDriven_model(model,n_epochs):

    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = []

    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3) # Adam optimizer

    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=50, verbose=True)

    for epoch in range(1,n_epochs+1):
        ###################
        # train the model #
        ###################
        model.train() # prep model for training

        for id_batch, (x_batch, y_batch) in enumerate(dataloader_DataDriven_train):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            NN_output = model(x_batch) # the predicted values x̃ are obtained by passing the input vector to the PIRNN model

            # Data driven loss term
            loss1 = torch.mean((NN_output[:, :, :] - y_batch[:, :, :])**2)  # use mean squared error
            """
            # # Compute the loss term corresponding to initial conditions


            # Compute the physics-driven loss term
            C_A0 = x_batch[:, :, 2] * std_X[2] + mean_X[2] + C_A0s
            Q = x_batch[:, :, 3] * std_X[3] + mean_X[3] + Q_s

            NN_output = NN_output * std_y.to(device) + mean_y.to(device) + torch.from_numpy(np.array([C_As, T_s])).float().to(device)

            dCA_first = (NN_output[:, 1:2, 0] - NN_output[:, 0:1, 0]) / (t_step*10)
            dT_first = (NN_output[:, 1:2, 1] - NN_output[:, 0:1, 1]) / (t_step*10)

            dCA_center = (NN_output[:, 2:, 0] - NN_output[:, :-2, 0]) / (2 * t_step*10)
            dT_center = (NN_output[:, 2:, 1] - NN_output[:, :-2, 1]) / (2 * t_step*10)

            dCA_last = (NN_output[:, -1:, 0] - NN_output[:, -2:-1, 0]) / (t_step*10)
            dT_last = (NN_output[:, -1:, 1] - NN_output[:, -2:-1, 1]) / (t_step*10)


            dCA = torch.cat((dCA_first, dCA_center, dCA_last), 1)
            dT = torch.cat((dT_first, dT_center, dT_last), 1)

            lossCA = dCA - F / V * (C_A0 - NN_output[:, :, 0]) + k_0 * torch.exp(-E / (R * NN_output[:, :, 1])) * NN_output[:, :, 0]**2
            lossCA = torch.mean(lossCA**2)

            lossT = dT - F / V * (T_0 - NN_output[:, :, 1]) + delta_H / (rho_L * C_p) * k_0 * torch.exp(-E / (R * NN_output[:, :, 1])) * NN_output[:, :, 0]**2 - Q / (rho_L * C_p * V)
            lossT = torch.mean(lossT**2)

            # print('loss1, lossCA, lossT', loss1, lossCA, lossT)
            # backpropagate joint loss
            loss = 10 * loss1 + 1e-1 * lossCA + 1e-5 * lossT # add all loss terms together
            """
            loss = loss1
            loss.backward()
            optimizer.step()
            # record training loss
            train_losses.append(loss.item())

    #     ######################
    #     # validate the model #
    #     ######################
    #     model.eval() # prep model for evaluation
    #     for val_batch,(x_valbatch, y_valbatch) in enumerate(dataloader_DataDriven_val):
    #         x_valbatch, y_valbatch = x_valbatch.to(device), y_valbatch.to(device)  # use valiadation data
    #         NN_output = model(x_valbatch)
    #         # Data driven loss term

    #         loss1 = torch.mean((NN_output[:, :, :] - y_valbatch[:, :, :])**2)
    #         """
    #         # Compute the physics-driven loss term
    #         C_A0 = x_valbatch[:, :, 2] * std_X[2] + mean_X[2] + C_A0s
    #         Q = x_valbatch[:, :, 3] * std_X[3] + mean_X[3] + Q_s

    #         NN_output = NN_output * std_y.to(device) + mean_y.to(device) + torch.from_numpy(np.array([C_As, T_s])).float().to(device)

    #         dCA_first = (NN_output[:, 1:2, 0] - NN_output[:, 0:1, 0]) / (t_step*10)
    #         dT_first = (NN_output[:, 1:2, 1] - NN_output[:, 0:1, 1]) / (t_step*10)

    #         dCA_center = (NN_output[:, 2:, 0] - NN_output[:, :-2, 0]) / (2 * t_step*10)
    #         dT_center = (NN_output[:, 2:, 1] - NN_output[:, :-2, 1]) / (2 * t_step*10)

    #         dCA_last = (NN_output[:, -1:, 0] - NN_output[:, -2:-1, 0]) / (t_step*10)
    #         dT_last = (NN_output[:, -1:, 1] - NN_output[:, -2:-1, 1]) / (t_step*10)


    #         dCA = torch.cat((dCA_first, dCA_center, dCA_last), 1)
    #         dT = torch.cat((dT_first, dT_center, dT_last), 1)

    #         lossCA = dCA - F / V * (C_A0 - NN_output[:, :, 0]) + k_0 * torch.exp(-E / (R * NN_output[:, :, 1])) * NN_output[:, :, 0]**2
    #         lossCA = torch.mean(lossCA**2)

    #         lossT = dT - F / V * (T_0 - NN_output[:, :, 1]) + delta_H / (rho_L * C_p) * k_0 * torch.exp(-E / (R * NN_output[:, :, 1])) * NN_output[:, :, 0]**2 - Q / (rho_L * C_p * V)
    #         lossT = torch.mean(lossT**2)

    #         # backpropagate joint loss
    #         loss = 10 * loss1 + 1e-1 * lossCA + 1e-5 * lossT # add all loss terms together
    #         """
    #         loss = loss1
    #         valid_losses.append(loss.item())

    #     # print training/validation statistics
    #     # calculate average loss over an epoch
    #     train_loss = np.average(train_losses)
    #     valid_loss = np.average(valid_losses)
    #     avg_train_losses.append(train_loss)
    #     avg_valid_losses.append(valid_loss)

    #     epoch_len = len(str(n_epochs))

    #     print_msg = (f'[{epoch}/{n_epochs}] ' +
    #                  f'train_loss: {train_loss:.5f} ' +
    #                  f'valid_loss: {valid_loss:.5f}')

    #     print(print_msg)
    #     # clear lists to track next epoch
    #     train_losses = []
    #     valid_losses = []

    #     # early_stopping needs the validation loss to check if it has decresed,
    #     # and if it has, it will make a checkpoint of the current model
    #     early_stopping(valid_loss, model)

    #     if early_stopping.early_stop:
    #         print("Early stopping")
    #         break

    # # load the last checkpoint with the best model
    # model.load_state_dict(torch.load('checkpoint.pt'))

    return  model, avg_train_losses, avg_valid_losses

In [462]:
from keras.layers import Dense, SimpleRNN, LSTM
import tensorflow as tf
from keras import Model, regularizers, activations
import pickle

class Model(tf.keras.layers.Layer):

    def __init__(self):
        super(Model, self).__init__()

        self.layer_1 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_2 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_3 = Dense(2, activation='linear')

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        x = self.layer_3(x)
        return x

model = Model()

# Load the Keras model from the pickle file
with open('/content/drive/MyDrive/Meta-Learning/CSTR+Batach+PFR/model_transfer_cstr_batch_pfr.sav', 'rb') as f:
    keras_model = pickle.load(f)

weights = keras_model.get_weights()
for item in weights:
  print(np.array(item).shape)

(4, 64)
(64, 64)
(64,)
(64, 64)
(64, 64)
(64,)
(64, 2)
(2,)


In [463]:
model_PINN = RNN(4, 2, [64, 64], 2)

# Assign the weights to the PyTorch model
with torch.no_grad():
    # Assign kernel weights
    model_PINN.rnn.weight_ih_l0.data = torch.tensor(weights[0].T)
    # Assign recurrent weights
    model_PINN.rnn.weight_hh_l0.data = torch.tensor(weights[1].T)
    # Assign bias
    model_PINN.rnn.bias_hh_l0.data = torch.tensor(weights[2])
    # Assign bias
    model_PINN.rnn.bias_ih_l0.fill_(0)
    # Assign kernel weights
    model_PINN.rnn1[0].weight_ih_l0.data = torch.tensor(weights[3].T)
    # Assign recurrent weights
    model_PINN.rnn1[0].weight_hh_l0.data = torch.tensor(weights[4].T)
    # Assign bias
    model_PINN.rnn1[0].bias_hh_l0.data = torch.tensor(weights[5])
    # Assign bias
    model_PINN.rnn1[0].bias_ih_l0.fill_(0)
    # Assign output weight
    model_PINN.output_layer.weight.data = torch.tensor(weights[6].T)
    # Assign output bias
    model_PINN.output_layer.bias.data = torch.tensor(weights[7])

In [34]:
seed = 0
rng = np.random.RandomState(seed)
sample_idx = rng.choice(len(RNN_input), size=10)
x_train = RNN_input[sample_idx]
y_train = RNN_output[sample_idx]

X_DataDriven_train = torch.from_numpy(x_train).float()
y_DataDriven_train = torch.from_numpy(y_train).float()

dataset_DataDriven_train = TensorDataset(X_DataDriven_train,y_DataDriven_train)
dataloader_DataDriven_train = DataLoader(dataset_DataDriven_train, batch_size=256, shuffle=True)

# Train the PIRNN model
model_PINN = RNN(4, 2, [64, 64], 2)
model_PINN.to(device)
print(model_PINN)

n_epochs = 500
DataDriven_model, train_loss, valid_loss = train_DataDriven_model(model_PINN, n_epochs)

RNN(
  (rnn): RNN(4, 64, batch_first=True)
  (rnn1): ModuleList(
    (0): RNN(64, 64, batch_first=True)
  )
  (output_layer): Linear(in_features=64, out_features=2, bias=True)
)


In [35]:
torch.save(DataDriven_model,'DataDriven.pkl')

In [36]:
"""
Test PIRNN model
"""
model_PINN = torch.load('DataDriven.pkl')
model_PINN.to(device)
print(model_PINN)

y_test_error = list()
model_PINN.eval()
for id_batch, (x_batch, y_batch) in enumerate(dataloader_total_test):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    NN_output = model_PINN(x_batch)
    MSE_loss= torch.mean((NN_output - y_batch)**2)
    y_test_error.append(MSE_loss.item())

print(f"mean error is {np.mean(y_test_error)}, std is {np.std(y_test_error)}")

RNN(
  (rnn): RNN(4, 64, batch_first=True)
  (rnn1): ModuleList(
    (0): RNN(64, 64, batch_first=True)
  )
  (output_layer): Linear(in_features=64, out_features=2, bias=True)
)
mean error is 0.11215837299823761, std is 0.0
